# AIAp Miniproject 2

## Content

- [Imports](#imports)

## Imports

In [ ]:
from keras import utils as keras_utils

## Dataset

### b) Citation and description

### c) Splitting into training and test data

In [ ]:

keras_utils.image_dataset_from_directory(directory="./data/raw/Architectual_Heritage_Elements_Dataset_128(creative_commons)", image_size=(128, 128))

train_val_imgs, test_images, train_val_lables, test_labels = model_selection.train_test_split()

## Exploratory data analysis

## Exploratory data analysis

## Architecture 1 (underfitting)

## Architecture 2 (overfitting)

## Architecture 2 (optimized)

## Quantification of the model performance

## Discussion